In [ ]:
# !apt install mongodb >log
# !service mongodb start
!pip install redis

     |████████████████████████████████| 81kB 5.5MB/s 


In [ ]:
!wget https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz
!gunzip amazon-meta.txt.gz

--2021-06-30 23:58:29--  https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210807517 (201M) [application/x-gzip]
Saving to: ‘amazon-meta.txt.gz’

amazon-meta.txt.gz  100%[===================>] 201.04M  5.91MB/s    in 31s     

2021-06-30 23:59:00 (6.46 MB/s) - ‘amazon-meta.txt.gz’ saved [210807517/210807517]



In [ ]:
def limpeza(text): #Criando estrutura de dados para o arquivo da amazon
  text = text.split('\n')
  # print(text)
  flag = 0
  vet = {}
  for words in text:
    # print(words)
    #words = words.rstrip()

    if words.startswith('Id:'):
      vet['id'] = words.split()[1].strip()

    elif words.startswith('ASIN:'):
      vet['asin'] = words.split('ASIN:')[1].strip()
        
    elif words.startswith('  title:'):
      vet['title'] = words.split('title: ')[1].strip()

    elif words.startswith('  group:'):
      vet['group'] = words.split('group:')[1].strip()

    elif words.startswith('  salesrank:'):
      vet['salesrank'] = words.split('salesrank:')[1].strip()

    elif words.startswith('  similar:'):
      if words.strip()[9] != 0:
        vet['similar'] = words.split()[2:]
      
    elif words.startswith('  categories:'):
      vet['categories'] = []

    elif words.startswith('   |'):
      categories = words.split('|')
      if len(categories) == 1:
          pass
      else:
          vet['categories'].append(categories[len(categories)-1])

    elif words.startswith('  reviews:'):
      vet['avg_rating'] =  words.split(': ')[4]
      vet['reviews'] = []

    elif words.startswith('    2') or words.startswith('    1'):
      aux = words.split()
      vet['reviews'].append({'data':aux[0], 'cliente':aux[2], 
                             'rating':aux[4], 'votes':aux[6], 'helpful':aux[8]})
  #print(vet)      
  return vet

def delimited(file, delimiter, bufsize=4096):
  buf = ''
  while True:
    newbuf = file.read(bufsize)
    if not newbuf:
      yield buf
      return
    buf += newbuf
    lines = buf.split(delimiter)
    for line in lines[:-1]:
      yield line
    buf = lines[-1]


In [ ]:
!sudo apt-get -qq update
!sudo apt-get -qq -y install redis-server
!pip install redis
!redis-server --daemonize yes
!ps aux | grep redis-server
!pip install simplejson

import redis
import json

r = redis.Redis(host='localhost', port=6379, db=0)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libjemalloc1.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../libjemalloc1_3.6.0-11_amd64.deb ...
Unpacking libjemalloc1 (3.6.0-11) ...
Selecting previously unselected package redis-tools.
Preparing to unpack .../redis-tools_5%3a4.0.9-1ubuntu0.2_amd64.deb ...
Unpacking redis-tools (5:4.0.9-1ubuntu0.2) ...
Selecting previously unselected package redis-server.
Preparing to unpack .../redis-server_5%3a4.0.9-1ubuntu0.2_amd64.deb ...
Unpacking redis-server (5:4.0

In [ ]:
data = list(delimited(open('amazon-meta.txt', 'r'), '\n\n'))

In [ ]:
#Limpeza dos dados 
cont = 0
vetor = []
for produto in data:
  if (not produto.startswith('#') and not produto == 'Total'):
    x = limpeza(produto)
    vetor.append(x)


In [ ]:
#Inserindo dentro do banco de dados
cont = 0
for i in vetor:
  cont+=1
  if cont > 1:
    # print(i[])
    try:
      r.set(i['asin'],json.dumps(i))
    except:
      print(i)
   # products.insert_one(i)
# vetor[1]

{}


In [ ]:
r.get('0738700797')

b'{"id": "2", "asin": "0738700797", "title": "Candlemas: Feast of Flames", "group": "Book", "salesrank": "168596", "similar": ["0738700827", "1567184960", "1567182836", "0738700525", "0738700940"], "categories": ["Wicca[12484]", "Witchcraft[12486]"], "avg_rating": "4.5", "reviews": [{"data": "2001-12-16", "cliente": "A11NCO6YTE4BTJ", "rating": "5", "votes": "5", "helpful": "4"}, {"data": "2002-1-7", "cliente": "A9CQ3PLRNIR83", "rating": "4", "votes": "5", "helpful": "5"}, {"data": "2002-1-24", "cliente": "A13SG9ACZ9O5IM", "rating": "5", "votes": "8", "helpful": "8"}, {"data": "2002-1-28", "cliente": "A1BDAI6VEYMAZA", "rating": "5", "votes": "4", "helpful": "4"}, {"data": "2002-2-6", "cliente": "A2P6KAWXJ16234", "rating": "4", "votes": "16", "helpful": "16"}, {"data": "2002-2-14", "cliente": "AMACWC3M7PQFR", "rating": "4", "votes": "5", "helpful": "5"}, {"data": "2002-3-23", "cliente": "A3GO7UV9XX14D8", "rating": "4", "votes": "6", "helpful": "6"}, {"data": "2002-5-23", "cliente": "A1GI

#1) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação


In [ ]:
p = json.loads(r.get('0738700797'))

v = sorted(p['reviews'], key=lambda x: (int(x['helpful']), int(x['rating'])),reverse=True)[0:5]
v = sorted(v, key=lambda x: (x['rating']),reverse=True)

print("5 comentários mais úteis ordenados pela maior avaliação")
for i in v:
  print(i)

print("\n5 comentários mais úteis ordenados pela menor avaliação")
v = sorted(v, key=lambda x: (x['rating']))
for i in v:
  print(i)

5 comentários mais úteis ordenados pela maior avaliação
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23', 'cliente': 'A1GIL64QK68WKL', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}

5 comentários mais úteis ordenados pela menor avaliação
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23

#2) Dado um produto, listar os produtos similares com maiores vendas do que ele

In [ ]:
produto = json.loads(r.get('0738700797'))
similares = produto['similar']

def clean(p):
  p.pop('similar', None)
  p.pop('categories', None)
  p.pop('reviews', None)
  p.pop('group', None)
  p.pop('avg_rating', None)

for i in similares:
  p_similar = json.loads(r.get(i))
 
  if int(produto['salesrank']) < int(p_similar['salesrank']):
    clean(p_similar)
    print(p_similar)


{'id': '454888', 'asin': '0738700827', 'title': 'Ostara: Customs, Spells & Rituals for the Rites of Spring', 'salesrank': '311254'}
{'id': '411179', 'asin': '1567182836', 'title': 'Beltane: Springtime Rituals, Lore and Celebration', 'salesrank': '280541'}


#3) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

In [ ]:
def media(text): #media de diaas
  vet = {'media':[]}
  rate = []
  cont = 1
  for review in text['reviews']:
    rate.append(float(review['rating']))
    if cont == 1:
      vet['media'].append({'media_evolucao': float(review['rating'])})       
    else:
      vet['media'].append({'media_evolucao': (sum(rate)) / cont}) 

    cont+=1
  return vet['media']

produto = json.loads(r.get('0738700797'))
x = media(p)
cont = 0


for key, value in p.items():
  #print(key,value)
  if key == 'reviews':
    for i in value:
      print(i, x[cont])
      cont+=1


{'data': '2001-12-16', 'cliente': 'A11NCO6YTE4BTJ', 'rating': '5', 'votes': '5', 'helpful': '4'} {'media_evolucao': 5.0}
{'data': '2002-1-7', 'cliente': 'A9CQ3PLRNIR83', 'rating': '4', 'votes': '5', 'helpful': '5'} {'media_evolucao': 4.5}
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'} {'media_evolucao': 4.666666666666667}
{'data': '2002-1-28', 'cliente': 'A1BDAI6VEYMAZA', 'rating': '5', 'votes': '4', 'helpful': '4'} {'media_evolucao': 4.75}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'} {'media_evolucao': 4.6}
{'data': '2002-2-14', 'cliente': 'AMACWC3M7PQFR', 'rating': '4', 'votes': '5', 'helpful': '5'} {'media_evolucao': 4.5}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'} {'media_evolucao': 4.428571428571429}
{'data': '2002-5-23', 'cliente': 'A1GIL64QK68WKL', 'rating': '5', 'votes': '8', 'helpful': '8'} {'media_evolucao': 4.5}
{'data': '20

#4) Listar os 10 produtos líderes de venda em categoria

In [ ]:
vet = []
cursor = '0'
while cursor != 0:
  cursor, keys = r.scan(cursor=cursor, count=100)
  for key in keys:
    produto = json.loads(r.get(key))
    try:
      if "Criminology[11005]" in produto['categories']:
        vet.append(produto)
    except:
      pass

new = sorted(vet, key=lambda x : x['salesrank'], reverse=True)[0:20]

def clean(p):
  p.pop('similar', None)
  p.pop('asin', None)
  p.pop('reviews', None)
  p.pop('group', None)
  p.pop('avg_rating', None)
  p.pop('categories', None)

cont = 0
for i in new:
  if cont % 2 == 0:
    clean(i)
    print(i)
  cont+=1

{'id': '517628', 'title': 'Prosecuting and Defending Health Care Fraud Cases', 'salesrank': '997450'}
{'id': '9374', 'title': 'With Liberty for Some: 500 Years of Imprisonment in America', 'salesrank': '995575'}
{'id': '295', 'title': 'Inventing Criminology: Essays on the Rise of Homo Criminalis (S U N Y Series in Deviance and Social Control)', 'salesrank': '994146'}
{'id': '192060', 'title': 'Studying and Preventing Homicide : Issues and Challenges', 'salesrank': '990132'}
{'id': '54392', 'title': 'Violence and Psychopathy', 'salesrank': '986537'}
{'id': '528480', 'title': 'Have a Seat, Please', 'salesrank': '984757'}
{'id': '384723', 'title': "Punishment and Democracy: 3 Strikes and You're Out in California (Studies in Crime and Public Policy                                         X)", 'salesrank': '983569'}
{'id': '531043', 'title': 'Sins of the City: The Real Los Angeles Noir', 'salesrank': '97707'}
{'id': '315844', 'title': 'Gun Violence : The Real Costs', 'salesrank': '972863'}


#5) Listar os 10 produtos com a maior média de ratings por grupo (Group) de produtos

In [ ]:
#p = json.loads(r.get('2'))

vet = []
cursor = '0'
while cursor != 0:
  cursor, keys = r.scan(cursor=cursor, count=100)
  for key in keys:
    produto = json.loads(r.get(key))
    try:
      group = produto['group']
      if group == 'Book':
        vet.append(produto)
    except:
      pass

new = sorted(vet, key=lambda x : x['avg_rating'], reverse=True)[0:10]

def clean(p):
  p.pop('similar', None)
  p.pop('asin', None)
  p.pop('reviews', None)
  p.pop('group', None)
  p.pop('avg_rating', None)
  p.pop('categories', None)

cont = 0
for i in new:
  print(i)
  # if cont % 2 == 0:
  #   clean(i)
  #   print(i)
  # cont+=1

{'id': '417386', 'asin': '1580130119', 'title': 'The Colors of My Jewish Year', 'group': 'Book', 'salesrank': '632178', 'similar': ['158013047X', '1580130496'], 'categories': ['Judaism[3120]', 'Jewish Life[12563]'], 'avg_rating': '5', 'reviews': [{'data': '2001-1-30', 'cliente': 'A3QZCA4LTTVGAD', 'rating': '5', 'votes': '1', 'helpful': '1'}]}
{'id': '445959', 'asin': '074603007X', 'title': 'First Thousand Words in French Sticker Book (Picture Word Books Series)', 'group': 'Book', 'salesrank': '434755', 'similar': ['0794502849', '0746027680', '0794502857', '0746037090', '0071421416'], 'categories': ['Vocabulary & Spelling[3187]'], 'avg_rating': '5', 'reviews': [{'data': '1999-11-28', 'cliente': 'A2DLJSJOHF1F3T', 'rating': '5', 'votes': '30', 'helpful': '30'}, {'data': '2001-7-25', 'cliente': 'A2LNC45164E5I8', 'rating': '5', 'votes': '13', 'helpful': '13'}, {'data': '2001-11-4', 'cliente': 'A2YW1TAPJLQ0ST', 'rating': '5', 'votes': '10', 'helpful': '10'}]}
{'id': '525975', 'asin': '096247

#6) Listar a 5 categorias de produto com a maior média geral de ratings 

In [ ]:
def clean(p):
  p.pop('similar', None)
  p.pop('asin', None)
  p.pop('reviews', None)
  p.pop('group', None)
  p.pop('title', None)
  p.pop('id', None)
  p.pop('salesrank', None)
  return p

vet = {}
cursor = '0'
cont = 0
while cursor != 0:
  cursor, keys = r.scan(cursor=cursor, count=100)
  for key in keys:
    produto = json.loads(r.get(key))
    
    try:
      avg = produto['avg_rating']
      if avg == '5':
        #vet.append(produto)
        produto = clean(produto)
        print(produto)
        cont+=1
      if cont == 5:
        break
    except:
      pass
  if cont == 5:
    break
    

    

{'categories': ['Judaism[3120]', 'Jewish Life[12563]'], 'avg_rating': '5'}
{'categories': ['Vocabulary & Spelling[3187]'], 'avg_rating': '5'}
{'categories': ['General[67188]', 'CDs $7 - $10[519728]', 'CDs $7 - $10[661690]', 'CDs $7 - $10[735046]', 'All Bargain Titles[883658]', 'All Bargain Titles[884224]', 'All Bargain Titles[884974]'], 'avg_rating': '5'}
{'categories': ['Pop Rock[284720]', 'Pop Rock[510558]', 'Pop Rock[520284]'], 'avg_rating': '5'}
{'categories': ['General[12642]', 'General[12667]'], 'avg_rating': '5'}


#7) Listar os 10 clientes que mais fizeram comentários por grupo (Group) de produto

In [ ]:
group = 'Book'

vet = {}

cursor = '0'
while cursor != 0:
  cursor, keys = r.scan(cursor=cursor, count=100)
  for key in keys:
    produto = json.loads(r.get(key))
    try:
      if produto['group'] == group:
        for cliente in produto['reviews']:
          if cliente['cliente'] in vet:
            vet[cliente['cliente']] = vet[cliente['cliente']] + 1
          else:
            vet[cliente['cliente']] = 1
    except:
      pass

x = sorted(vet.items(), key=lambda x: x[1], reverse=True)[0:10]
print(group)
for j in x:
  print(j)


Book
('ATVPDKIKX0DER', 643185)
('A3UN6WX5RRO2AG', 154531)
('A14OJS0VWMOSWO', 9589)
('AFVQZQ8PW0L', 5441)
('A1K1JW1C5CUSUZ', 3562)
('A2NJO6YE954DBH', 2055)
('A3QVAKVRAH657N', 1651)
('A1NATT3PN24QWY', 1535)
('A1D2C0WDCSHUWZ', 1508)
('A2ODBHT4URXVXQ', 1469)
